# Arbeitslosenrate mit Bildungslevel
Autor: André Amann (5839020), 12.12.21


---

## Aufgaben
<ol start="0">
  <li> Daten Sammeln und herunterladen</li>
  <li> Daten über die Bildungsrate und Arbeitslosigkeit betrachten</li>
  <li> Lineare Regression.</li>
  <li> RSS vs. polyfit.</li>
  <li> Bootstrap zwischen Bildungsrate und Arbeitslosenrate</li>
  <li> Bootstrap regressions.</li>
  <li> Hypothesentest der Pearson Korrelation.</li>
  <li> Ergebnisse und Fazit.</li>
</ol>

# Aufgabe 1

---------------------------------------------

## Ergebnisse
- Daten und Spaltennamen sind verschieden .
- Eurostat Daten beinhalten mehr Länder als OECD Daten
- Eurostat Daten sind ein wenig anders bei Deutschland.

## Nächster Schritt:
- Spaltennamen normalisieren
- Deutschland von eurostat extrahieren.
- Alle drei Datensätze in einen mergen.
- Speichern des gemergten Datensatzes in `unemployment_edu.csv`.

In [1]:
eurostat = Path("rawdata", "eurostat")
eurostat_data = read_raw_data(low_file=eurostat.joinpath("EuroStat_Low_Germany.csv"),
                              middle_file=eurostat.joinpath("EuroStat_Middle_Germany.csv"),
                              high_file=eurostat.joinpath("EuroStat_High_Germany.csv"))
print("EUROSTAT\n=============")
display(eurostat_data["Low"])
print(eurostat_data["Low"]["GEO"].unique())
df = eurostat_data["Low"]
eurostat_germany_low = df.loc[df["GEO"] == "Germany (until 1990 former territory of the FRG)"]
eurostat_germany_low["TIME"] = eurostat_germany_low["TIME"].astype(int)
eurostat_germany_low["Value"] = eurostat_germany_low["Value"].astype(float)
plt.xlabel("Years")
plt.ylabel("Unemployes population in \%")
plt.plot("TIME", "Value", data=eurostat_germany_low, label="Eurostat | Germany | Low")
oecd = Path("rawdata", "oecd")
oecd_data = read_raw_data(low_file=oecd.joinpath("OECD_Low_Germany_USA.csv"),
                          middle_file=oecd.joinpath("OECD_Middle_Germany_USA.csv"),
                          high_file=oecd.joinpath("OECD_High_Germany_USA.csv"))
print("\nOECD\n=============")
display(oecd_data["Low"])
print(oecd_data["Low"]["LOCATION"].unique())
df = oecd_data["Low"]
oecd_data_low = df.loc[df["LOCATION"] == "DEU"]
oecd_data_low["TIME"] = oecd_data_low["TIME"].astype(int)
oecd_data_low["Value"] = oecd_data_low["Value"].astype(float)
plt.plot("TIME", "Value", data=oecd_data_low, label="OECD | Germany | Low")
plt.title("OECD vs eurostat (Germany, low education)")
plt.legend()
plt.show()


## Mergen und säubern der Daten

Mergen der Datensätze eurostat und OECD und sichern des normalisierten Datensatzes in `unemployment_edu.csv`.

In [2]:
clean_data = merge_and_clean_data(eurostat_data, oecd_data)
display(clean_data)
clean_data.to_csv("unemployment_edu.csv")
plot_data(clean_data, "USA")
plot_data(clean_data, "Germany")

## Pearson Koeffizient

Schauen ob eine Korrelation zwischen Arbeitslosenrate mit Personen niedriger, mittlerer und hoher Bildung in einem Land besteht.

## Ergebnisse
- Alle Arbeitslosenraten korrelieren miteinander (pearson > 0)
  - Die Kurven passen zueinander bis zu einem gewissen Punkt -> Arbeitslosenraten sind auch Zeitabhängig.

In [3]:
usa1 = pearson_r(clean_data["USA - Low"], clean_data["USA - Middle"])
usa2 = pearson_r(clean_data["USA - Low"], clean_data["USA - High"])
print("USA", usa1, usa2)germ1 = pearson_r(clean_data["Germany - Low"], clean_data["Germany - Middle"])
germ2 = pearson_r(clean_data["Germany - Low"], clean_data["Germany - High"])
print("Germany", germ1, germ2)


USA 0.9691689314586223 0.9628178522127707

Germany 0.9666440312284225 0.911975489026696


# Aufgabe 2 und 3

-------------------------------

## Lineare Regression

Ausführen einer Linearen Regression mit `np.polyfit`.

## Ergebnisse
- Eindimensionale lineare Regression passt nicht zu USA Daten. 
  - 3D passt besser.
- Lineare Regression passt zu den Deutschland Daten.
 - Slope: Arbeitslos Delta in Prozent für 1 Jahr.
 - Intercept: Akkumulierte Arbeitslos Prozentsatz im Jahr 0.
- Wenn Slope positiv ist, steigt die Arbeitslosenrate im Laufe der Zeit.
- Wenn Slope negativ ist, fällt die Arbeitsloserate im Laufe der Zeit.

In [4]:
plot_linear_regression(clean_data["Year"].dt.year, clean_data["USA - Low"], "USA | Low education")
plot_linear_regression(clean_data["Year"].dt.year, clean_data["USA - Middle"], "USA | Middle education")
plot_linear_regression(clean_data["Year"].dt.year, clean_data["USA - High"], "USA | High education")
plot_linear_regression(clean_data["Year"].dt.year, clean_data["Germany - Low"], "Germany | Low education")
plot_linear_regression(clean_data["Year"].dt.year, clean_data["Germany - Middle"], "Germany | Middle education")
plot_linear_regression(clean_data["Year"].dt.year, clean_data["Germany - High"], "Germany | High education")

## Vergleich RSS mit np.polyfit

RSS berechnen:
- Leeren Arry für slopes erzeugen.
- RSS mit slopes Array berechnen : `np.sum((y - a * x - b) ** 2)`.

## Ergebnisse
- RSS ist ziemlich hoch in den USA und niedrig in Deutschland.
  - USA Datensatz passt nicht so gut wie Deutschland zum linearen Regressionsmodell.


In [5]:

for country in ["USA", "Germany"]:
    for education in ["Low", "Middle", "High"]:
        linear_regression_vs_rss(clean_data["Year"].dt.year, clean_data[F"{country} - {education}"], F"{country} | {education} education")
        slope_vs_rss(clean_data["Year"].dt.year, clean_data[F"{country} - {education}"])

# Aufgabe 4

----------------

## Paaren von Bootstrap und Konfidenzintervallen

Ausführen von `pairs_bootstrap` um eine Schätzung für den Slope von Bildung und Arbeitslosenrate zu beschreiben.

## Methode
- Slopes mit `pairs_bootstrap` berechnen und mit `plt.hist` plotten.
- Konfidenzintervalle mit `np.percentile` berechnen.

## Notiz
Nur niedrige Bildungslevel Daten werden analysiert.

In [6]:
slopes, intercepts = pairs_bootstrap(clean_data["Year"].dt.year, clean_data["Germany - Low"], 1000)
plt.xlabel("Slopes")
plt.ylabel("PDF")
_ = plt.hist(slopes, bins=50, density=True)
print("Confidence interval: ", np.percentile(slopes, [2.5, 97.5]))

# Aufgabe 5

-------------------

## Bootstrap regressions

100 Zeilen von den Bootstrap werten von Aufgabe 4 werden geplottet

In [7]:
x = np.array([clean_data["Year"].dt.year.min(), clean_data["Year"].dt.year.max()])
for i in range(100):
    _ = plt.plot(x, slopes[i] * x + intercepts[i], linewidth=0.5, alpha=0.3, color="red")
plot_single_data(clean_data, "Germany - Low")

# Aufgabe 6

-------------

## Hypothesentest auf der Pearson Korrelation

Null Hypothese: Die Arbeitslosenrate eines Landes basiert <b>nicht</b> auf dem Bildungslevel.

## Methode
- Berechnen der Pearson Werte von Arbeitslosigkeit und Bildungsrate.
- Permutieren einiger Werte während die Bildungswerte unangetastet verbleiben.
- Für jede Permutation wird die Pearson Korrelation berechnet. Außerdem wird berechnet, wie viele der Permutierten Replikate einen Pearson Korrelationskoeffizient haben, der größer ist als der Beobachtete.
- Wenn der p Wert klein ist, wird die Hypothese: Die Arbeitslosenrate eines Landes basiert auf der Bildungsrate, angenommen (p<= 0,05).
- Wenn der p-Wert höher ist, wird die Null Hypothese angenommen.

## Ergebnisse
- P-Werte mit Daten desselben Landes sind sehr niedrig.
- Die Alternativ Hypothese wird angenommen: <b>Die Arbeitslosenrate eines Landes basiert auf der Bildungsrate</b>.



# Aufgabe 7

---------------

## Ergebnisse und Fazit

- Aufgaben 2 und 3:
  - Die Arbeitslosenrate in Deutschland wird kleiner im Laufe der Zeit.
  - Die Arbeitslosenrate in den USA wird größer für mittlere und hohe Bildungsraten.
- Aufgabe 6:
  - Die Arbeitslosenrate eines Landes basiert auf dem Bildungsniveau.
  - Es gibt keine Verbindung der Bildungsraten zwischen USA und Deutschland.

## A/B Test
Definitione eines A/B Test : <b>2 seperate Gruppen interagieren mit zwei Varianten eines Systems, das nur in einem Punkt unterschiedlich ist</b>
Das gegebene System von Arbeitslosen- und Bildungsrate passt <b>nicht</b> zu den Kriterien eines A/B Test. Begründet werden kann das durch die stetigkeit des Bildungssystems und die Existenz mehrerer Faktoren, die das Ergebnis beeinflussen.
 
## Bonferroni Test
Wenn ein Test gegen die Null-Hypothese durchgeführt wird, gibt es immer die Möglichkeit eines falsch-positiven Ergebnisses. Um dies zu verhindern kann eine Bonferroni Korrektion angewendet werden.
Bsp: die Error Rate 10% eträgt und 5 tests durchgeführt werden würde die die BK 0,1 / 5 = 0,2 betragen.